In [10]:
#import libraries
import json
import os
import xml.etree.ElementTree as ET
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [6]:
#load dataset splits
with open('archive/splits.json', 'r') as json_splits:
  data = json.load(json_splits)

#split data into train and test
train_imgs = data["train"]
test_imgs = data["test"]

In [4]:
#function to extract image path from xml file
def extract_image_path(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    #jpg path found on <path> tag
    path_element = root.find('path')
    if path_element is not None:
        return path_element.text
    return None

train_image_paths = []
for xml_file in train_imgs:
    xml_path = os.path.join('archive/annotated-images/', xml_file)
    image_path = extract_image_path(xml_path)
    
    if image_path:
        image_full_path = os.path.join('archive/annotated-images/', image_path)
        train_image_paths.append(image_full_path)

In [7]:
train_image_paths = []
test_image_paths = []

old_string = '/CSE-800/Thesis/2020-01-23/dataset'
new_string = 'archive/annotated-images'

def extract_image_path(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    path_element = root.find('path')
    return path_element.text

#get image paths for train_imgs
for xml_file in train_imgs:
    xml_path = os.path.join('archive/annotated-images/', xml_file)
    image_path = extract_image_path(xml_path)
    if image_path:
        image_full_path = os.path.join('archive/annotated-images/', image_path)
        train_image_paths.append(image_full_path)
train_image_paths = [path.replace(old_string, new_string) for path in train_image_paths]

#get image paths for test_imgs
for xml_file in test_imgs:
    xml_path = os.path.join('archive/annotated-images/', xml_file)
    image_path = extract_image_path(xml_path)
    if image_path:
        image_full_path = os.path.join('archive/annotated-images/', image_path)
        test_image_paths.append(image_full_path)
test_image_paths = [path.replace(old_string, new_string) for path in test_image_paths]


In [8]:
#display image at index 0 on each split
Image.open(train_image_paths[0]).show()
Image.open(test_image_paths[0]).show()

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(720, 720, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#TRAIN_IMAGE_PATHS IS A LIST OF PATHS. TO WORK IN HISOTRY IT MUST GET THE ACTUAL IMAGE
history = model.fit(train_image_paths, epochs=25)